In [55]:
#Depedencies
from dash import Dash, html, dcc, callback, Output, Input
import pandas as pd
import plotly.express as px
from stats_can import StatsCan
import dash_bootstrap_components as dbc
import datetime as dt
sc = StatsCan()

In [56]:
sc = StatsCan(data_folder="Resources")

# Preping Tech Use

In [57]:
advance_tech_use_df = sc.table_to_df("27-10-0367-01")

In [59]:
advance_tech_use_df.rename(columns={'North American Industry Classification System (NAICS)':'NAICS', 
                                    'REF_DATE': 'Date',
                                    'VALUE':'Percentage',
                                    'Advanced or emerging technologies': 'Advanced Technology'}, 
                          inplace=True)
advance_tech_use_df['Date'] = advance_tech_use_df['Date'].dt.year.astype(str)

In [60]:
plot_data=advance_tech_use_df.groupby(['Date','GEO','NAICS','Enterprise size', 'Advanced Technology'], observed=False)['Percentage'].mean()

In [61]:
tech_plot_data_df=pd.DataFrame(plot_data)

In [62]:
tech_plot_data_df.reset_index(inplace=True)

In [98]:
tech_plot_data_df.dropna(inplace=True)

In [101]:
tech_list = ['Artificial intelligence (AI)', 'Biotechnology',
             'Blockchain technologies', 'Clean technologies',
             'Design or information control technologies',
             'Geomatics or geospatial technologies',
             'Integrated Internet of Things (IoT) systems',
             'Material handling, supply chain or logistics technologies',
             'Nanotechnology', 'Other types of advanced technologies',
             'Processing or fabrication technologies',
             'Business intelligence technologies',
             'Security or advanced authentication systems',
             'Additive manufacturing', 'Virtual, mixed and augmented reality',
             'Quantum technology', 'Robotics']

In [102]:
tech_list

['Artificial intelligence (AI)',
 'Biotechnology',
 'Blockchain technologies',
 'Clean technologies',
 'Design or information control technologies',
 'Geomatics or geospatial technologies',
 'Integrated Internet of Things (IoT) systems',
 'Material handling, supply chain or logistics technologies',
 'Nanotechnology',
 'Other types of advanced technologies',
 'Processing or fabrication technologies',
 'Business intelligence technologies',
 'Security or advanced authentication systems',
 'Additive manufacturing',
 'Virtual, mixed and augmented reality',
 'Quantum technology',
 'Robotics']

# Preping Reasons

In [65]:
reason_not_used_df = sc.table_to_df("27-10-0368-01")

In [66]:
reason_not_used_df

,REF_DATE,GEO,DGUID,North American Industry Classification System (NAICS),Enterprise size,Reasons for not using advanced or emerging technologies,UOM,UOM_ID,SCALAR_FACTOR,SCALAR_ID,VECTOR,COORDINATE,VALUE,STATUS,SYMBOL,TERMINATED,DECIMALS
0,2017-01-01,Canada,2021A000011124,"Total, all surveyed industries","Total, all enterprise sizes",Not convinced of economic benefit,Percent,239,units,0,v1047710650,1.1.1.1,18.2,A,NaN,NaN,1
1,2017-01-01,Canada,2021A000011124,"Total, all surveyed industries","Total, all enterprise sizes",Difficulty in obtaining financing,Percent,239,units,0,v1047710651,1.1.1.2,5.8,A,NaN,NaN,1
2,2017-01-01,Canada,2021A000011124,"Total, all surveyed industries","Total, all enterprise sizes",High cost of advanced or emerging technologies,Percent,239,units,0,v1047710652,1.1.1.3,18.2,A,NaN,NaN,1
3,2017-01-01,Canada,2021A000011124,"Total, all surveyed industries","Total, all enterprise sizes",Investment not necessary for continuing operat...,Percent,239,units,0,v1047710653,1.1.1.4,37.0,A,NaN,NaN,1
4,2017-01-01,Canada,2021A000011124,"Total, all surveyed industries","Total, all enterprise sizes",Lack of technical skills required to support t...,Percent,239,units,0,v1047710654,1.1.1.5,10.1,A,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31815,2022-01-01,Rest of Canada excluding Alberta,NaN,"Administrative and support, waste management a...","Total, all enterprise sizes",Difficulty in integrating new advanced technol...,Percent,239,units,0,v1586548706,7.95.1.10,1.0,A,NaN,NaN,1
31816,2022-01-01,Rest of Canada excluding Alberta,NaN,"Administrative and support, waste management a...","Total, all enterprise sizes",Impact of the COVID-19 pandemic,Percent,239,units,0,v1586548709,7.95.1.13,0.2,A,NaN,NaN,1
31817,2022-01-01,Rest of Canada excluding Alberta,NaN,"Administrative and support, waste management a...","Total, all enterprise sizes",Not convinced of the environmental benefits,Percent,239,units,0,v1586548710,7.95.1.14,0.0,E,NaN,NaN,1
31818,2022-01-01,Rest of Canada excluding Alberta,NaN,"Administrative and support, waste management a...","Total, all enterprise sizes",Other reasons for not adopting or using advanc...,Percent,239,units,0,v1586548707,7.95.1.11,0.7,A,NaN,NaN,1


In [67]:
reason_not_used_df.rename(columns={'North American Industry Classification System (NAICS)':'NAICS', 
                                   'Reasons for not using advanced or emerging technologies':'Reason Not Adopted',
                                   'REF_DATE': 'Date',
                                   'VALUE': 'Percentage'}, 
                          inplace=True)
reason_not_used_df['Date'] = reason_not_used_df['Date'].dt.year.astype(str)

In [68]:
plot_data=reason_not_used_df.groupby(['Date','GEO','NAICS','Enterprise size', 'Reason Not Adopted'], observed=False)['Percentage'].mean()

In [69]:
reason_plot_data_df=pd.DataFrame(plot_data)

In [70]:
reason_plot_data_df.reset_index(inplace=True)

In [71]:
reason_plot_data_df.dropna(inplace=True)
reason_plot_data_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 26423 entries, 5475 to 114070
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   Date                26423 non-null  object  
 1   GEO                 26423 non-null  category
 2   NAICS               26423 non-null  object  
 3   Enterprise size     26423 non-null  object  
 4   Reason Not Adopted  26423 non-null  object  
 5   Percentage          26423 non-null  float64 
dtypes: category(1), float64(1), object(4)
memory usage: 1.2+ MB


In [105]:
app = Dash(__name__, external_stylesheets=[dbc.themes.YETI])
markdown_reason = '''
#### Description

Percentage of enterprises that did not adopt or use advanced technologies for specific reasons, 
by North American Industry Classification System (NAICS) code and enterprise size, based on a one-year observation period. 
Reasons for not adopting or using advanced technologies include not being convinced of economic benefit; 
difficulty in obtaining financing; high cost of advanced technologies; investment not necessary for continuing operations; 
lack of technical skills required to support this type of investment; organizational culture too inflexible; 
decisions made by parent, affiliates or subsidiary businesses; lack of technical support or services (from consultants or vendors); 
lack of information regarding advanced technology; difficulty in integrating new advanced technologies with existing systems, standards, and processes;
other reasons for not adopting or using advanced technologies; and adoption or use of advanced technologies not applicable to this business’s activities.  
[Adapated from Statistics Canada Table: 27-10-0367-01 Released: 2024-04-30](https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?pid=2710036701)  
*This does not constitute an endorsement by Statistics Canada of this product.*
'''
markdown_adoption = '''
#### Description  

Percentage of enterprises that used specific types of advanced or emerging technologies, by North American Industry Classification System (NAICS) code and 
enterprise size, based on a one-year observation period. Advanced technologies include material handling, supply chain or logistics technologies; design or 
information control technologies; processing or fabrication technologies; clean technologies; security or advanced authentication systems; 
business intelligence technologies; and other types of advanced technologies. Emerging technologies include nanotechnology, biotechnology, 
geomatics or geospatial technologies, artificial intelligence (AI), integrated Internet of Things (IoT) systems, blockchain technologies, 
and other types of emerging technologies.  
[Adapted from Statistics Canada Table: 27-10-0368-01 Released date: 2024-04-30](https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?pid=2710036801)    
*This does not constitute an endorsement by Statistics Canada of this product.*
'''
app.layout = html.Div([
    
    html.H1(children='Technology Adoption in Canada', style={'textAlign':'center'}),
    dcc.Dropdown(reason_plot_data_df.GEO.unique(), 'Canada', id='dropdown-selection-geo'),
    dcc.Dropdown(reason_plot_data_df.NAICS.unique(),'Total, all surveyed industries', id= 'dropdown-selection-naics'),
    dcc.Dropdown(reason_plot_data_df['Enterprise size'].unique(), 'Total, all enterprise sizes', id = 'dropdown-selection-enterprisesize'),
   
    # dcc.RadioItems(reason_plot_data_df.REF_DATE.unique(), id = 'radio-selection-ref-date'),
    
    dcc.Graph(id='graph-2-content'),
    dcc.Markdown(children=markdown_adoption),
    dcc.Graph(id='graph-1-content'),
    dcc.Markdown(children=markdown_reason),
    dcc.Graph(id='graph-3-content'),
    html.Label('Technology: ', id= 'label-selection-technology'),
    dcc.Dropdown(tech_plot_data_df['Advanced Technology'].unique(), tech_list, id = 'dropdown-selection-technology', multi=True, ),
])

@callback(
    Output('graph-1-content', 'figure'),
    Output('graph-2-content', 'figure'),
    Output('graph-3-content', 'figure'),
    Input('dropdown-selection-geo', 'value'),
    Input('dropdown-selection-naics', 'value'),
    Input('dropdown-selection-enterprisesize', 'value'),
    Input('dropdown-selection-technology', 'value'),
    # Input('radio-selection-ref-date', 'value')
)
def update_graph(getgeo, getnaics, getsize, gettech):
    df1 = reason_plot_data_df.loc[(reason_plot_data_df.GEO==getgeo) &
                (reason_plot_data_df.NAICS == getnaics) & 
                    (reason_plot_data_df['Enterprise size'] == getsize) 
                    # (reason_plot_data_df.REF_DATE == getdate)
    ]
    df2 = tech_plot_data_df.loc[(tech_plot_data_df.GEO==getgeo) &
                (tech_plot_data_df.NAICS == getnaics) & 
                    (tech_plot_data_df['Enterprise size'] == getsize) &
                    (tech_plot_data_df['Advanced Technology'].isin(tech_list))
                    # (reason_plot_data_df.REF_DATE == getdate)
    ]
    df3 = tech_plot_data_df.loc[(tech_plot_data_df.GEO==getgeo) &
                (tech_plot_data_df.NAICS == getnaics) & 
                    (tech_plot_data_df['Enterprise size'] == getsize) &
                    (tech_plot_data_df['Advanced Technology'].isin(gettech))
    ]
    df2 = df2.sort_values(by='Percentage', ascending=True)
    fig_1 = px.scatter(df1, 
                       y='Reason Not Adopted', 
                       x='Percentage', 
                       size='Percentage', 
                       color='Date', 
                       # color_discrete_sequence=["red", "green", "blue"],
                       title= f'Reasons for Not Adopted - {getgeo} - {getnaics}',
                       )
    fig_2 = px.bar(df2, 
                   y='Advanced Technology', 
                   x='Percentage', 
                   color='Date',
                   # color_discrete_sequence=["red", "green", "blue"],
                   title = f'Technologies Adopted - {getgeo} - {getnaics}',
                   height = 600,
                  )
    fig_3 = px.bar(df3, 
                   y='Percentage', 
                   x='Date', 
                   color='Advanced Technology', 
                   barmode='group',
                   title = f'Technologies Adopted Over Time - {getgeo} - {getnaics}',
                  ) 
    
    return fig_1, fig_2, fig_3
    
if __name__ == '__main__':
    app.run(jupyter_mode='jupyterlab', debug=None)